In [1]:
import pandas as pd
import numpy as np


#visualization
import matplotlib as mpl
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBaclend.figure_format = 'svg'


#preprocessing tools

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer #equal to get dummies
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder


In [2]:
diamonds = pd.read_csv('./data/diamonds_train.csv.zip')
diamonds_predict = pd.read_csv('./data/diamonds_predict.csv')

In [3]:
TARGET = 'price' 


NUM_FEATS = ['carat', 'depth', 'x', 'y']
CAT_FEATS = ['cut', 'color', 'clarity']
FEATS = NUM_FEATS + CAT_FEATS

In [4]:
numeric_transformer = \
Pipeline(steps=[('imputer', SimpleImputer(strategy='mean')),
               ('scaler', StandardScaler())])

In [5]:
categorical_transformer = \
Pipeline(steps=[('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
               ('onehot', OneHotEncoder(handle_unknown='ignore'))])#handle unkown dont compute unkown values

In [6]:
preprocessor = \
ColumnTransformer(transformers=[('num', numeric_transformer, NUM_FEATS),
                               ('cat', categorical_transformer, CAT_FEATS)])

In [7]:
preprocessor

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer', SimpleImputer()),
                                                 ('scaler', StandardScaler())]),
                                 ['carat', 'depth', 'x', 'y']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='missing',
                                                                strategy='constant')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['cut', 'color', 'clarity'])])

In [10]:
pd.DataFrame(data=preprocessor.fit_transform(diamonds)).head()

#.todense(), to be able to see the dataframe properly and not tabular manner

,0
0,"(0, 0)\t0.86700559845042\n (0, 1)\t0.452018..."
1,"(0, 0)\t-1.004557489479606\n (0, 1)\t0.8710..."
2,"(0, 0)\t-0.18443433858892058\n (0, 1)\t2.61..."
3,"(0, 0)\t-0.815298300812525\n (0, 1)\t1.4298..."
4,"(0, 0)\t0.4674584223754707\n (0, 1)\t-0.875..."


In [11]:
from sklearn.model_selection import train_test_split

In [12]:
diamonds_train, diamonds_test = train_test_split(diamonds)

In [13]:
print(diamonds_train.shape)
print(diamonds_test.shape)

(30341, 10)
(10114, 10)


In [14]:
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor

model = Pipeline(steps=[('preprocessor',preprocessor),
                       ('regressor', RandomForestRegressor())])

In [15]:
model.fit(diamonds_train[FEATS], diamonds_train[TARGET]);

In [16]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score

In [17]:
y_test = model.predict(diamonds_test[FEATS])
y_train = model.predict(diamonds_train[FEATS])


In [18]:
print(f"Test error: {mean_squared_error(y_pred=y_test,y_true=diamonds_test[TARGET], squared=False)}")
print(f"Train error: {mean_squared_error(y_pred=y_train,y_true=diamonds_train[TARGET], squared=False)}")

Test error: 562.9923596872192
Train error: 215.9009710422966


In [19]:
print(f"test mean sq error: {r2_score(y_pred=y_test, y_true=diamonds_test[TARGET])}")
print(f"train mean sq error: {r2_score(y_pred=y_train, y_true=diamonds_train[TARGET])}")

test mean sq error: 0.9803151960274704
train mean sq error: 0.9970655306924456


In [20]:
scores = cross_val_score(model,
                        diamonds[FEATS],
                        diamonds[TARGET],
                        scoring='neg_root_mean_squared_error',
                        cv=5, n_jobs=-1)

In [21]:
np.mean(-scores)

566.2715200812811

In [22]:
from sklearn.model_selection import RandomizedSearchCV 

In [23]:
param_grid = {
    'preprocessor__num__imputer__strategy': ['mean', 'median'],
    'regressor__n_estimators': [16, 32, 64, 128, 256, 512],
    'regressor__max_depth': [2, 4, 8, 16],
}

grid_search = RandomizedSearchCV(model, 
                                 param_grid, 
                                 cv=5, 
                                 verbose=10, 
                                 scoring='neg_root_mean_squared_error', 
                                 n_jobs=-1,
                                 n_iter=32)

grid_search.fit(diamonds[FEATS], diamonds[TARGET])

Fitting 5 folds for each of 32 candidates, totalling 160 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   42.5s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   45.1s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  5.1min
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  5.7min
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  7.2min
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:  8.6min
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed: 10.7min
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed: 14.0min
/home/emmanuel/miniconda3/envs/ironhack_env/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed: 35.

RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('preprocessor',
                                              ColumnTransformer(transformers=[('num',
                                                                               Pipeline(steps=[('imputer',
                                                                                                SimpleImputer()),
                                                                                               ('scaler',
                                                                                                StandardScaler())]),
                                                                               ['carat',
                                                                                'depth',
                                                                                'x',
                                                                                'y']),
                                 

In [24]:
grid_search.best_params_

{'regressor__n_estimators': 512,
 'regressor__max_depth': 16,
 'preprocessor__num__imputer__strategy': 'median'}

In [25]:
grid_search.best_score_

-560.3341893537306

In [26]:
y_pred = grid_search.predict(diamonds_predict[FEATS])

In [27]:
submission_df = pd.DataFrame({'id': diamonds_predict['id'],
                             'price': y_pred})

In [28]:
submission_df.head()

,id,price
0,0,2999.310483
1,1,5392.324094
2,2,9152.521648
3,3,3962.968332
4,4,1684.079451


In [30]:
submission_df.price.clip(370,19000, inplace=True)

In [31]:
submission_df.to_csv('./submissions/noon_08_09.csv', index=False)